<left>
    <img src="http://epecora.com.br/DataFiles/PPGOLD_PPGMNE.png" width="800" alt="PPGOLD Data Science: INTRO"  />
</left>

# **Combinatorial Optimization - PPGMNE:**  

## Taks

* Program a random seacrh for the TSP problem

Steps:

1. Read the file (Code already given)
2. Program a function to compute the objetive function, having as input a solution and a distance matrix
3. Program a function to randomly generate a solution
4. Randomly generate `2*n` solutions and print the best


---

Marina's Results


In [1]:
# Teste Marina

import os
import pandas as pd
import numpy as np
import random


In [2]:
def read_tsp_file(filename):
    if os.path.exists(filename):
        data = pd.read_csv(filename)
        return data
    else:
        print(f"File {filename} not found.")
        return None

In [3]:

def read_tsp_file(filename):
    if os.path.exists(filename):
        data = pd.read_csv(filename)
        return data
    else:
        print(f"File {filename} not found.")
        return None


def read_all_tsp_files(directory):
    tsp_files = {}
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            full_path = os.path.join(directory, filename)
            tsp_files[filename] = read_tsp_file(full_path)
    return tsp_files


def calculate_total_distance(solution, distance_matrix):
    total_distance = 0
    n = len(solution)
    for i in range(n - 1):
        total_distance += distance_matrix[solution[i], solution[i + 1]]

    total_distance += distance_matrix[solution[-1], solution[0]]
    return total_distance


def generate_random_solution(n):
    solution = list(range(n))
    random.shuffle(solution)
    return solution

def random_search(tsp_distance, num_solutions):
    n = tsp_distance.shape[0]
    S_best = None
    OF_best = np.inf

    for _ in range(num_solutions):

        S_random = generate_random_solution(n)
        

        OF_random = calculate_total_distance(S_random, tsp_distance)
        

        if OF_random < OF_best:
            S_best = S_random
            OF_best = OF_random

    return S_best, OF_best

directory = r"C:\Users\55489\Documents\GitHub\NMUN-7124\TSP"

tsp_files = read_all_tsp_files(directory)

for filename, tsp_data in tsp_files.items():
    if tsp_data is not None:
        print(f"\nProcessing file: {filename}")
        tsp_data.drop(columns=['X','Y'], inplace=True)
        tsp_distance = tsp_data.to_numpy()


        n = tsp_distance.shape[0]
        

        best_solution, best_distance = random_search(tsp_distance, num_solutions=2*n)

        print("Best solution found:", best_solution)
        print("Total distance of the best solution:", best_distance)




Processing file: TSP_05_001.csv
Best solution found: [1, 2, 0, 3, 4]
Total distance of the best solution: 2.1556714736878737

Processing file: TSP_05_002.csv
Best solution found: [0, 2, 4, 1, 3]
Total distance of the best solution: 2.30903843771977

Processing file: TSP_05_003.csv
Best solution found: [4, 2, 0, 1, 3]
Total distance of the best solution: 2.1614912139348434

Processing file: TSP_05_004.csv
Best solution found: [3, 0, 1, 4, 2]
Total distance of the best solution: 2.5249438853630033

Processing file: TSP_05_005.csv
Best solution found: [0, 4, 3, 2, 1]
Total distance of the best solution: 3.1899277603396117

Processing file: TSP_10_001.csv
Best solution found: [7, 3, 6, 0, 4, 8, 1, 5, 2, 9]
Total distance of the best solution: 3.6625912706100983

Processing file: TSP_10_002.csv
Best solution found: [6, 3, 4, 7, 5, 0, 2, 9, 8, 1]
Total distance of the best solution: 4.0516105278733

Processing file: TSP_10_003.csv
Best solution found: [0, 5, 8, 4, 7, 6, 1, 3, 2, 9]
Total di